<a href="https://colab.research.google.com/github/FernandoValencia-DS/ML_Salud/blob/main/Fine_Tunning_XLMRoBERTa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Integrantes
### Alejandro Orozco 2402036-7727
### Fernando Valencia 2401899-7727
### Carlos Botero 2400879-7727
### Brandon Rivas 2400430-7727

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets transformers
!pip install seqeval
! pip install -U datasets evaluate
!pip install -U huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=be970adcf0570213bbf1d7d80c5b915b09a882c29c271f71d38ce73cdff54b76
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resol

In [ ]:
from datasets import DatasetDict, Dataset, Features, Sequence, Value, ClassLabel
from collections import defaultdict
from pathlib import Path
import pandas as pd

# 📄Rutas de Archivos

Se definen las rutas de los tres archivos _(train, test y valid)_  con los que se hará el ajuste del modelo.

In [ ]:
# Se definen los nombres de las rutas (paths) de los archivos .bio
rutas_archivos = {
    "train": "/content/drive/MyDrive/Analitica en Salud/sentences_train.csv",
    "test": "/content/drive/MyDrive/Analitica en Salud/sentences_test.csv",
    "valid": "/content/drive/MyDrive/Analitica en Salud/sentences_dev.csv"
}

# 🗂️ Carga de datos desde archivos CSV
Este bloque permite cargar datos anotados en formato **CSV**, donde cada fila contiene un token, su etiqueta y el identificador de la oración.

# 🔧 Función cargar_csv_conll()
Lee un archivo CSV con columnas como `Sentence #`, `Word` y `Tag`.

- Completa los IDs de oración que estén vacíos (NaN).

- Elimina filas que no tengan token o etiqueta.

- Agrupa los tokens y etiquetas por cada oración.

- Devuelve un diccionario con dos listas: `tokens` y `ner_tags`.

# 🔍 Detección de etiquetas únicas
A partir de las etiquetas presentes en el set de entrenamiento, se genera una lista ordenada llamada `LABELS` que contiene todas las etiquetas utilizadas en el esquema BIO.

# 🏗️ Creación del DatasetDict
Define un esquema de datos (`features`) que especifica:

- `tokens`: secuencias de cadenas (`palabras`).

- `ner_tags`: secuencias de clases (`ClassLabel`) basadas en las etiquetas detectadas.

Esto permite que las etiquetas en texto (`B_CANCER_CONCEPT`, `I_CANCER_CONCEPT`, `O`) se conviertan en números (0, 1, 2) que el modelo puede procesar.

Luego, los datos de entrenamiento, validación y prueba se transforman en objetos `Dataset` aplicando este esquema, y se agrupan en un `DatasetDict` listo para el entrenamiento.

In [ ]:
# Función para cargar y agrupar datos desde CSV tipo CoNLL
def cargar_csv_conll(path):
    df = pd.read_csv(path)

    # Rellenar IDs de oración (NaN) y eliminar filas con palabras o etiquetas vacías
    df["Sentence #"] = df["Sentence #"].ffill()
    df = df.dropna(subset=["Word", "Tag"])

    # Convertir a string explícitamente para evitar errores con floats
    df["Word"] = df["Word"].astype(str)
    df["Tag"] = df["Tag"].astype(str)

    # Agrupar tokens y etiquetas por oración
    grouped = df.groupby("Sentence #")
    tokens = grouped["Word"].apply(list)
    ner_tags = grouped["Tag"].apply(list)

    return {"tokens": tokens.tolist(), "ner_tags": ner_tags.tolist()}

# Cargar datasets
train_data = cargar_csv_conll(rutas_archivos["train"])
val_data = cargar_csv_conll(rutas_archivos["valid"])
test_data = cargar_csv_conll(rutas_archivos["test"])

# Detectar etiquetas únicas para el esquema BIO
LABELS = sorted({etiqueta for lista in train_data["ner_tags"] for etiqueta in lista})

# Crear esquema de features
features = Features({
    "tokens": Sequence(Value("string")),
    "ner_tags": Sequence(ClassLabel(names=LABELS))
})

# Construir DatasetDict completo con casting a esquema
dataset_dict = DatasetDict({
    "train": Dataset.from_dict(train_data).cast(features),
    "validation": Dataset.from_dict(val_data).cast(features),
    "test": Dataset.from_dict(test_data).cast(features)
})

# Mostrar información básica
print("Etiquetas detectadas:", LABELS)
print("\nEjemplo del dataset:")
print(dataset_dict["train"][0])

Casting the dataset:   0%|          | 0/9788 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/2758 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/2496 [00:00<?, ? examples/s]

Etiquetas detectadas: ['B_CANCER_CONCEPT', 'B_CHEMOTHERAPY', 'B_DATE', 'B_DRUG', 'B_FAMILY', 'B_FREQ', 'B_IMPLICIT_DATE', 'B_INTERVAL', 'B_METRIC', 'B_OCURRENCE_EVENT', 'B_QUANTITY', 'B_RADIOTHERAPY', 'B_SMOKER_STATUS', 'B_STAGE', 'B_SURGERY', 'B_TNM', 'I_CANCER_CONCEPT', 'I_DATE', 'I_DRUG', 'I_FAMILY', 'I_FREQ', 'I_IMPLICIT_DATE', 'I_INTERVAL', 'I_METRIC', 'I_OCURRENCE_EVENT', 'I_SMOKER_STATUS', 'I_STAGE', 'I_SURGERY', 'I_TNM', 'O']

Ejemplo del dataset:
{'tokens': ['Abuela', 'materna', 'con', 'cancer', 'de', 'mama', 'a', 'los', '70', 'años', '.'], 'ner_tags': [4, 19, 29, 0, 16, 16, 29, 29, 10, 8, 29]}


In [ ]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 9788
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 2758
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 2496
    })
})

#⚙️ Configuración del modelo
Se definen los parámetros principales del modelo:

- Tarea: ner (Reconocimiento de Entidades Nombradas).

- Modelo preentrenado: xlm-roberta-base, adecuado para texto en español.

- Tamaño de batch: 32, utilizado durante el entrenamiento y evaluación.



In [ ]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "xlm-roberta-base"
batch_size=32

In [ ]:
x = dataset_dict["train"].features[f"{task}_tags"].feature.names
print(x)

['B_CANCER_CONCEPT', 'B_CHEMOTHERAPY', 'B_DATE', 'B_DRUG', 'B_FAMILY', 'B_FREQ', 'B_IMPLICIT_DATE', 'B_INTERVAL', 'B_METRIC', 'B_OCURRENCE_EVENT', 'B_QUANTITY', 'B_RADIOTHERAPY', 'B_SMOKER_STATUS', 'B_STAGE', 'B_SURGERY', 'B_TNM', 'I_CANCER_CONCEPT', 'I_DATE', 'I_DRUG', 'I_FAMILY', 'I_FREQ', 'I_IMPLICIT_DATE', 'I_INTERVAL', 'I_METRIC', 'I_OCURRENCE_EVENT', 'I_SMOKER_STATUS', 'I_STAGE', 'I_SURGERY', 'I_TNM', 'O']


# 🔧 Corrección y actualización de las etiquetas
Se realiza una corrección en las etiquetas para pasar del formato con guión bajo (`B_CANCER_CONCEPT`) al formato estándar con guión medio (`B-CANCER_CONCEPT`), utilizado en el esquema BIO.

Luego, se redefine el esquema de datos (`features`) con las etiquetas corregidas y se aplica a todas las particiones del dataset (`train`, `validation` y `test`). Esto garantiza que las etiquetas estén correctamente mapeadas y que el modelo las interprete de manera adecuada.

In [ ]:
from datasets import ClassLabel, Sequence, Value, Features

etiquetas_corregidas = [
    'B-CANCER_CONCEPT', 'B-CHEMOTHERAPY', 'B-DATE', 'B-DRUG', 'B-FAMILY',
    'B-FREQ', 'B-IMPLICIT_DATE', 'B-INTERVAL', 'B-METRIC', 'B-OCURRENCE_EVENT',
    'B-QUANTITY', 'B-RADIOTHERAPY', 'B-SMOKER_STATUS', 'B-STAGE', 'B-SURGERY', 'B-TNM',
    'I-CANCER_CONCEPT', 'I-DATE', 'I-DRUG', 'I-FAMILY', 'I-FREQ',
    'I-IMPLICIT_DATE', 'I-INTERVAL', 'I-METRIC', 'I-OCURRENCE_EVENT',
    'I-SMOKER_STATUS', 'I-STAGE', 'I-SURGERY', 'I-TNM', 'O'
]


# Redefinir las features con etiquetas corregidas
features_corregidas = Features({
    "tokens": Sequence(Value("string")),
    f"{task}_tags": Sequence(ClassLabel(names=etiquetas_corregidas))
})

# Recast todos los splits
for split in dataset_dict:
    dataset_dict[split] = dataset_dict[split].cast(features_corregidas)

Casting the dataset:   0%|          | 0/9788 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/2758 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/2496 [00:00<?, ? examples/s]

In [ ]:
x = dataset_dict["train"].features[f"{task}_tags"].feature.names
print(x)

['B-CANCER_CONCEPT', 'B-CHEMOTHERAPY', 'B-DATE', 'B-DRUG', 'B-FAMILY', 'B-FREQ', 'B-IMPLICIT_DATE', 'B-INTERVAL', 'B-METRIC', 'B-OCURRENCE_EVENT', 'B-QUANTITY', 'B-RADIOTHERAPY', 'B-SMOKER_STATUS', 'B-STAGE', 'B-SURGERY', 'B-TNM', 'I-CANCER_CONCEPT', 'I-DATE', 'I-DRUG', 'I-FAMILY', 'I-FREQ', 'I-IMPLICIT_DATE', 'I-INTERVAL', 'I-METRIC', 'I-OCURRENCE_EVENT', 'I-SMOKER_STATUS', 'I-STAGE', 'I-SURGERY', 'I-TNM', 'O']


# 🔗 Tokenización y alineación de etiquetas
Se utiliza el tokenizador del modelo **xlm-roberta-base**.

Dado que algunas palabras pueden dividirse en subtokens, se implementa la función `tokenize_and_align_labels`, que:

Asigna las etiquetas solo al primer subtoken de cada palabra.

Marca los subtokens adicionales con -100 para que sean ignorados durante el entrenamiento.

Este proceso se aplica a todas las particiones del dataset mediante `.map()`, dejando el dataset tokenizado y listo para el entrenamiento del modelo.



In [ ]:
# --------------------------------------------------------------------------
# Tokenizador + alineación de etiquetas con B/I/O correctas
# --------------------------------------------------------------------------
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "xlm-roberta-base",
    add_prefix_space=True          # evita la fragmentación de la 1.ª palabra
)

# Obtiene la lista completa de etiquetas desde el dataset
label_list = dataset_dict["train"].features["ner_tags"].feature.names
label2id   = {l: i for i, l in enumerate(label_list)}
id2label   = {i: l for i, l in enumerate(label_list)}

def tokenize_and_align_labels(examples):
    tok_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True
    )

    aligned = []
    for i, word_labels in enumerate(examples["ner_tags"]):
        word_ids = tok_inputs.word_ids(batch_index=i)
        prev_wid = None
        label_ids = []
        for wid in word_ids:
            if wid is None:                     # tokens especiales <s>, </s>, padding
                label_ids.append(-100)
            elif wid != prev_wid:               # primer sub-token de la palabra
                label_ids.append(word_labels[wid])          # B-XXX u O
            else:                               # sub-token interior
                lab_name = label_list[word_labels[wid]]     # p. ej. "B-CANCER_CONCEPT"
                if lab_name.startswith("B-"):
                    inside_name = "I-" + lab_name[2:]
                    inside_id   = label2id.get(inside_name, word_labels[wid])
                    label_ids.append(inside_id)             # I-XXX
                else:                       # si la palabra es "O", se queda en O
                    label_ids.append(word_labels[wid])
            prev_wid = wid
        aligned.append(label_ids)

    tok_inputs["labels"] = aligned
    return tok_inputs

# Re-mapear el dataset
tokenized_datasets = dataset_dict.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset_dict["train"].column_names     # opcional, limpia columnas viejas
)


Map:   0%|          | 0/9788 [00:00<?, ? examples/s]

Map:   0%|          | 0/2758 [00:00<?, ? examples/s]

Map:   0%|          | 0/2496 [00:00<?, ? examples/s]

In [ ]:
label_list = dataset_dict["train"].features[f"{task}_tags"].feature.names
label_list

['B-CANCER_CONCEPT',
 'B-CHEMOTHERAPY',
 'B-DATE',
 'B-DRUG',
 'B-FAMILY',
 'B-FREQ',
 'B-IMPLICIT_DATE',
 'B-INTERVAL',
 'B-METRIC',
 'B-OCURRENCE_EVENT',
 'B-QUANTITY',
 'B-RADIOTHERAPY',
 'B-SMOKER_STATUS',
 'B-STAGE',
 'B-SURGERY',
 'B-TNM',
 'I-CANCER_CONCEPT',
 'I-DATE',
 'I-DRUG',
 'I-FAMILY',
 'I-FREQ',
 'I-IMPLICIT_DATE',
 'I-INTERVAL',
 'I-METRIC',
 'I-OCURRENCE_EVENT',
 'I-SMOKER_STATUS',
 'I-STAGE',
 'I-SURGERY',
 'I-TNM',
 'O']

# 🔧 Carga del Tokenizer, Modelo y Data Collator para NER

Se cargan los componentes esenciales para ajustar el modelo a la tarea de NER:

- `AutoTokenizer`: convierte texto en tokens compatibles con el modelo.
- `AutoModelForTokenClassification`: modelo para clasificación de tokens, ajustado al número de etiquetas (`num_labels`) según `label_list`.
- `DataCollatorForTokenClassification`: gestiona el padding dinámico de los lotes (*batches*), asegurando que las entradas sean del mismo tamaño durante el entrenamiento.


In [ ]:
label_list = [
    "B-CANCER_CONCEPT","B-CHEMOTHERAPY","B-DATE","B-DRUG","B-FAMILY",
    "B-FREQ","B-IMPLICIT_DATE","B-INTERVAL","B-METRIC","B-OCURRENCE_EVENT",
    "B-QUANTITY","B-RADIOTHERAPY","B-SMOKER_STATUS","B-STAGE","B-SURGERY","B-TNM",
    "I-CANCER_CONCEPT","I-DATE","I-DRUG","I-FAMILY","I-FREQ","I-IMPLICIT_DATE",
    "I-INTERVAL","I-METRIC","I-OCURRENCE_EVENT","I-SMOKER_STATUS","I-STAGE",
    "I-SURGERY","I-TNM","O"
]

id2label = {i: l for i, l in enumerate(label_list)}
label2id = {l: i for i, l in enumerate(label_list)}


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

# ⚙️ Configuración de los Parámetros de Entrenamiento

Se definen los hiperparámetros y configuraciones necesarias para el fine-tuning del modelo en la tarea de NER sobre datos de próstata.

- **Nombre del experimento:** se genera dinámicamente combinando el nombre del modelo base (`model_roberta_base`) y la tarea (`task`).
- **`TrainingArguments`:** clase que gestiona los parámetros de entrenamiento:
  - `eval_strategy="epoch"`: evalúa el modelo al final de cada época.
  - `learning_rate=2e-5`: tasa de aprendizaje.
  - `per_device_train_batch_size` y `per_device_eval_batch_size`: tamaño del batch para entrenamiento y evaluación.
  - `num_train_epochs=5`: número de épocas.
  - `weight_decay=0.01`: regularización para evitar sobreajuste.
  - `push_to_hub=True`: permite subir automáticamente el modelo al Hugging Face Hub.
  - `hub_token`: token de autenticación personal necesario para subir el modelo al Hugging Face Hub desde el entorno local.


In [ ]:
from transformers import TrainingArguments

model_roberta_base = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_roberta_base}-finetuned-{task}-pulmon",
    eval_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    push_to_hub=True,
    report_to="none",
    hub_token=""
)

In [ ]:
try:
    from datasets import load_metric  # Para versiones antiguas
    metric = load_metric("seqeval")
except ImportError:
    from evaluate import load  # Para versiones nuevas
    metric = load("seqeval")

# 📊 Función de evaluación
Esta función calcula las métricas de desempeño del modelo en la tarea de NER.

- Convierte las predicciones en etiquetas mediante argmax.

- Filtra los tokens con la etiqueta -100 (Los tokens con la etiqueta O).

- Traduce los índices de las etiquetas a sus nombres usando label_list.

- Calcula precisión, recall, F1 y exactitud utilizando el objeto metric.

La salida es un diccionario con estas métricas, que se usa para evaluar el modelo durante el entrenamiento y validación.

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# 🚀 Definición del entrenador
Se instancia el objeto `Trainer`, que gestiona el proceso de entrenamiento y evaluación del modelo.

Parámetros principales:
- model: El modelo ajustado (XLM-RoBERTa adaptado para NER).

- args: Argumentos de configuración del entrenamiento (Definidos anteriormente)

- train_dataset: Conjunto de datos de entrenamiento.

- eval_dataset: Conjunto de datos de validación.

- data_collator: Función que gestiona el padding dinámico durante el entrenamiento.

- tokenizer: Tokenizador asociado al modelo.

- compute_metrics: Función para calcular las métricas de evaluación en cada epoch.

Este objeto simplifica el manejo de todo el ciclo de entrenamiento, validación y evaluación del modelo.



In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-21-1966647044.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.118330,0.903383,0.913481,0.908404,0.970126
2,0.430900,0.089423,0.916040,0.953722,0.934502,0.976849
3,0.430900,0.077327,0.926428,0.955734,0.940853,0.980184
4,0.087200,0.076385,0.927169,0.960541,0.943560,0.980441
5,0.063800,0.077236,0.925456,0.963112,0.943909,0.980387


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=1530, training_loss=0.19129380953857322, metrics={'train_runtime': 275.3787, 'train_samples_per_second': 177.719, 'train_steps_per_second': 5.556, 'total_flos': 2353944567252960.0, 'train_loss': 0.19129380953857322, 'epoch': 5.0})

# ✅ Conclusión del entrenamiento
Los resultados del entrenamiento muestran una evolución positiva del modelo a lo largo de las epochs:

Desde la primera epoch, el modelo alcanza un rendimiento alto, con un **F1 de 0.92** y una **accuracy de 97.2%**, lo que indica que el modelo aprende rápidamente patrones relevantes.

A partir de la segunda epoch, se observa una mejora continua en todas las métricas, especialmente en **F1**, que sube hasta **0.94**.

La pérdida de validación disminuye de manera consistente hasta estabilizarse en torno a **0.072–0.076**, lo que sugiere que el modelo generaliza correctamente y no presenta indicios significativos de sobreajuste.

Las métricas de **precisión y recall** se mantienen balanceadas, lo que es ideal para tareas de NER, donde tanto la detección correcta como la cobertura son importantes.

Buenas prácticas:

No uses test para tomar decisiones: Solo para la evaluación final

Usa validación para ajustes: Early stopping, learning rate, etc.

Guarda test para el final: Como si fuera datos "reales" que el modelo nunca ha visto

In [ ]:
test_metrics = trainer.evaluate(tokenized_datasets["test"])
print("\n" + "="*50)
print(f"Resultados finales en conjunto de test:")
print(f"F1-score: {test_metrics['eval_f1']:.3f}")
print(f"Precisión: {test_metrics['eval_precision']:.3f}")
print(f"Recall: {test_metrics['eval_recall']:.3f}")
print("="*50)


Resultados finales en conjunto de test:
F1-score: 0.934
Precisión: 0.907
Recall: 0.963


# 🧪 Resultados finales en el conjunto de test
El modelo mantiene un desempeño consistente en datos no vistos, confirmando su capacidad de generalización:

F1-score: 0.933

Precisión: 0.911

Recall: 0.957



In [ ]:
trainer.push_to_hub()

Uploading...:   0%|          | 0.00/1.13G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/FernandoValencia/xlm-roberta-base-finetuned-ner-pulmon/commit/bdbb0d21809b993bef491871dfc4a2c810f5cd9c', commit_message='End of training', commit_description='', oid='bdbb0d21809b993bef491871dfc4a2c810f5cd9c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/FernandoValencia/xlm-roberta-base-finetuned-ner-pulmon', endpoint='https://huggingface.co', repo_type='model', repo_id='FernandoValencia/xlm-roberta-base-finetuned-ner-pulmon'), pr_revision=None, pr_num=None)

In [ ]:
label_names =  dataset_dict["train"].features["ner_tags"].feature.names
label_names

['B-CANCER_CONCEPT',
 'B-CHEMOTHERAPY',
 'B-DATE',
 'B-DRUG',
 'B-FAMILY',
 'B-FREQ',
 'B-IMPLICIT_DATE',
 'B-INTERVAL',
 'B-METRIC',
 'B-OCURRENCE_EVENT',
 'B-QUANTITY',
 'B-RADIOTHERAPY',
 'B-SMOKER_STATUS',
 'B-STAGE',
 'B-SURGERY',
 'B-TNM',
 'I-CANCER_CONCEPT',
 'I-DATE',
 'I-DRUG',
 'I-FAMILY',
 'I-FREQ',
 'I-IMPLICIT_DATE',
 'I-INTERVAL',
 'I-METRIC',
 'I-OCURRENCE_EVENT',
 'I-SMOKER_STATUS',
 'I-STAGE',
 'I-SURGERY',
 'I-TNM',
 'O']

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'CANCER_CONCEPT': {'precision': np.float64(0.9023383768913342),
  'recall': np.float64(0.9521044992743106),
  'f1': np.float64(0.9265536723163842),
  'number': np.int64(689)},
 'CHEMOTHERAPY': {'precision': np.float64(0.9249492900608519),
  'recall': np.float64(1.0),
  'f1': np.float64(0.9610115911485774),
  'number': np.int64(456)},
 'DATE': {'precision': np.float64(0.9808673469387755),
  'recall': np.float64(0.9871630295250321),
  'f1': np.float64(0.9840051183621241),
  'number': np.int64(779)},
 'DRUG': {'precision': np.float64(0.9116022099447514),
  'recall': np.float64(0.9777777777777777),
  'f1': np.float64(0.9435310936383131),
  'number': np.int64(675)},
 'FAMILY': {'precision': np.float64(0.9798657718120806),
  'recall': np.float64(0.9931972789115646),
  'f1': np.float64(0.9864864864864865),
  'number': np.int64(147)},
 'FREQ': {'precision': np.float64(0.8603351955307262),
  'recall': np.float64(0.9565217391304348),
  'f1': np.float64(0.9058823529411766),
  'number': np.int64(

# 📊 Desempeño por entidad
El modelo muestra un alto desempeño general, con un F1 global de 0.937, una precisión de 0.919 y un recall de 0.957, lo que indica una buena capacidad para identificar entidades correctamente.

## 🔍 Análisis por entidad:
Entidades con desempeño sobresaliente, con F1 superiores al 0.95:

CHEMOTHERAPY (0.98)

DATE (0.98)

FAMILY (0.99)

METRIC (0.94)

QUANTITY (0.96)

STAGE (0.97)

TNM (0.96)

Buen desempeño en entidades clínicas clave como:

CANCER_CONCEPT: F1 de 0.92

DRUG: F1 de 0.94

RADIOTHERAPY: F1 de 0.95

Entidades con desempeño aceptable pero con margen de mejora:

OCURRENCE_EVENT: F1 de 0.80

SURGERY: F1 de 0.86

SMOKER_STATUS: F1 de 0.87

Entidad con bajo desempeño:

IMPLICIT_DATE: F1 de 0.46, probablemente debido a su baja frecuencia (26 muestras) y a la mayor complejidad para identificar fechas implícitas en texto clínico.